In [1]:
# !pip install flair

In [2]:
import sys
sys.path.append('../')

In [3]:
from Datasets.ThDatasetVISTEC import ThDatasetVISTEC
# maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-sample", name="VISTEC-sample")
maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-2021", name="VISTEC")

In [4]:
# from flair.embeddings import WordEmbeddings, FlairEmbeddings
# from flair.embeddings import StackedEmbeddings

# glove_embedding = WordEmbeddings('glove')
# flair_embedding_forward = FlairEmbeddings('news-forward')
# flair_embedding_backward = FlairEmbeddings('news-backward')

# stacked_embeddings = StackedEmbeddings([ glove_embedding, flair_embedding_forward, flair_embedding_backward,])
# sentence = Sentence('The grass is green .')

# # just embed a sentence using the StackedEmbedding as you would with any single embedding.
# stacked_embeddings.embed(sentence)

# # now check out the embedded tokens.
# for token in sentence:
#     print(token)
#     print(token.embedding.shape)

In [5]:
# # https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md

# from flair.data import Sentence
# from flair.tokenization import JapaneseTokenizer

# tokenizer = JapaneseTokenizer("janome")
# japanese_sentence = Sentence("私はベルリンが好き", use_tokenizer=tokenizer)

# print(japanese_sentence)

## Train LM

In [6]:
from flair.data import Dictionary
from flair.models import LanguageModel
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus

In [7]:
# # TEMPCHARS = chr(0x4E20)
# additionalTokenMapping = {}

In [8]:
# make an empty character dictionary
from flair.data import Dictionary
import collections
from tqdm import tqdm
from util import *
import pickle

def build_dictionary(path, dataset, alphabetized=False):
    char_dictionary = Dictionary()
    counter = collections.Counter()

    with tqdm(total=40000) as pbar:
        tokens = 0
        processed = 0
        for d in dataset.datasets["train"]:
            sent = d["sent"]
            if alphabetized:
                chars = alphabetize(sent)
            else:
                chars = list(sent)
            processed += 1
            tokens += len(chars)
            counter.update(chars)

            pbar.update(1)

    total_count = tokens

    sum = 0
    idx = 0
    
    additionTokens = {}
    nAdd = 0
    
    for letter, count in counter.most_common():
        sum += count
        percentile = (sum / total_count)

        # comment this line in to use only top X percentile of chars, otherwise filter later
        # if percentile < 0.00001: break
        
        if len(letter) > 1:
            newletter = chr(0x4E20 + nAdd)
            while newletter in counter:
                nAdd += 1
                newletter = chr(0x4E20 + nAdd)
            nAdd += 1
            
            additionTokens[letter] = newletter
            additionTokens[newletter] = letter
            
            letter = newletter
            
            
            
        char_dictionary.add_item(letter)
        idx += 1
    #     print('%d\t%s\t%7d\t%7d\t%f' % (idx, letter, count, sum, percentile))
    
    dict_name = "char_mappings"
    if alphabetized:
        dict_name += "_alph"
        
    with open(f'{path}/{dict_name}', 'wb') as f:
        mappings = {
            'idx2item': char_dictionary.idx2item,
            'item2idx': char_dictionary.item2idx
        }
        pickle.dump(mappings, f)
    
    print("Number of sentences", processed)
    print("Number of tokens", tokens)
    print("Number of addition tokens", len(additionTokens.keys()))
    
    return additionTokens

In [9]:
# additionTokens = build_dictionary("./Models", maindataset, alphabetized=False)

# # Number of sentences 39000
# # Number of tokens 10862818

In [10]:
# additionTokens = build_dictionary("./Models", maindataset, alphabetized=True)
# # Number of sentences 39000
# # Number of tokens 10278787

In [11]:
# with open(f'./Models/alphs_addition_tokens', 'wb') as f:
#     pickle.dump(additionTokens, f)

In [12]:
with open(f'./Models/alphs_addition_tokens', 'rb') as f:
    additionTokens = pickle.load(f)

In [13]:
# D = Dictionary.load_from_file('./Models/char_mappings_alph')

In [19]:
# for c in D.idx2item:
#     print(c.decode("utf-8"))

In [13]:
# import pickle
# dictionary = Dictionary.load_from_file('./Models/char_mappings')
# # dictionary = Dictionary.load_from_file('./Models/char_mappings_alph')
# for i, k in enumerate(dictionary.item2idx):
#     print(k.decode("utf-8"), dictionary.item2idx[k])
#     if i > 10:
#         break

In [14]:
def custom_alphabetize(sent):
    tokens = alphabetize(sent)
    
    newtokens = []
    for t in tokens:
        if len(t) > 1:
            newtokens.append(additionTokens[t])
        else:
            newtokens.append(t)
    
    return newtokens

custom_alphabetize("เมียกินปลา")

['ม', '丣', 'ก', 'ิ', 'น', 'ป', 'ล', 'า']

In [15]:
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus, TextDataset
import torch
import logging
log = logging.getLogger("flair")
import random

class MyTextDataset(TextDataset):
    def __init__(
        self,
        dataset,
        dictionary: Dictionary,
        alphabetized: bool, 
        expand_vocab: bool = False,
        forward: bool = True,
        split_on_char: bool = True,
        random_case_flip: bool = True,    #ignore
        document_delimiter: str = "\n", #ignore
        shuffle: bool = True,
    ):
        self.dataset = dataset
        self.dictionary = dictionary
        self.split_on_char = split_on_char
        self.forward = forward
        self.random_case_flip = random_case_flip
        self.expand_vocab = expand_vocab
        self.document_delimiter = document_delimiter
        self.shuffle = shuffle
        
        self.alphabetized = alphabetized

        self.files = []

    def __len__(self):
        return 1

    def __getitem__(self, index=0) -> torch.Tensor:
        """Tokenizes a text file on character basis."""
        lines = []
        for d in self.dataset:
            sent = d["sent"]

            if self.alphabetized:
                chars = custom_alphabetize(sent)
            else:
                chars = list(sent)
        
#             l = chars
            l = chars + [self.document_delimiter]
            lines.append(l)

        log.info(f"read text file with {len(lines)} lines")

        if self.shuffle:
            random.shuffle(lines)
            log.info("shuffled")

        if self.expand_vocab:
            for chars in lines:
                for char in chars:
                    self.dictionary.add_item(char)
                    
        nums = [self.dictionary.get_idx_for_item(char) for chars in lines for char in chars]
        ids = torch.tensor(nums, dtype=torch.long)
        
        if not self.forward:
            ids = ids.flip(0)
        return ids
    
class MyTextCorpus(TextCorpus):
    def __init__(
        self,
        datasets,
        dictionary: Dictionary,
        alphabetized: bool,
        forward: bool = True,
        character_level: bool = True,
    ):
        self.dictionary: Dictionary = dictionary
        self.forward = forward
        self.split_on_char = character_level
        self.alphabetized = alphabetized
        
        
        self.random_case_flip = True
        self.expand_vocab = False
        self.document_delimiter = "\n"
        self.shuffle = False
        
        splitmaping = {
            "train": "train",
            "valid": "validation",
            "test": "test",
        }
        
        for k in splitmaping:
            sp = splitmaping[k]
            d = MyTextDataset(
                datasets.datasets[sp],
                dictionary,
                alphabetized,
                forward = self.forward,
                split_on_char = self.split_on_char,
                expand_vocab = self.expand_vocab,
                random_case_flip = self.random_case_flip,
                document_delimiter=self.document_delimiter,
                shuffle=self.shuffle,
            )
            
            if sp =="train":
                setattr(self, k, d)
            else:
                setattr(self, k, d[0])

In [16]:
L = []
for d in maindataset.datasets["train"]:
    L.append(len(d["sent"]))

sum(L)/len(L)

278.5337948717949

In [17]:
import pickle
from Datasets.ThDatasetVISTEC import ThDatasetVISTEC

def train_model(maindataset, alphabetized):
    
    if alphabetized:
        dictionary = Dictionary.load_from_file('./Models/char_mappings_alph')
    else:
        dictionary = Dictionary.load_from_file('./Models/char_mappings')
    
    print("#Vocab:", len(dictionary.idx2item))
    
    # forward_lm
    corpus = MyTextCorpus(maindataset, dictionary, alphabetized=alphabetized, forward=True)
    language_model = LanguageModel(dictionary, True, hidden_size=128, nlayers=1)
    trainer = LanguageModelTrainer(language_model, corpus)
    
    model_dir = './Models/fwdLM'
    if alphabetized:
        model_dir += "_alph"
        
    trainer.train(model_dir, sequence_length=250, mini_batch_size=128, max_epochs=10)
    
    # backward_lm
    corpus = MyTextCorpus(maindataset, dictionary, alphabetized=alphabetized, forward=False)
    language_model = LanguageModel(dictionary, False, hidden_size=128, nlayers=1)
    trainer = LanguageModelTrainer(language_model, corpus)
    
    model_dir = './Models/bkwLM'
    if alphabetized:
        model_dir += "_alph"
        
    trainer.train(model_dir, sequence_length=280, mini_batch_size=128, max_epochs=20)

In [26]:
# maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-sample", name="VISTEC-sample")
maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-2021", name="VISTEC")
train_model(maindataset, alphabetized=False)

#Vocab: 165
2022-05-25 09:12:38,854 read text file with 1000 lines
2022-05-25 09:12:39,071 read text file with 10000 lines
2022-05-25 09:12:41,310 read text file with 39000 lines
2022-05-25 09:12:44,639 Sequence length is 250
2022-05-25 09:12:44,700 Split 1	 - (09:12:44)
2022-05-25 09:13:15,518 | split   1/  1 |   100/  340 batches | ms/batch 308.16 | loss 3.5836 | ppl 36.0039
2022-05-25 09:13:46,413 | split   1/  1 |   200/  340 batches | ms/batch 308.94 | loss 2.6938 | ppl 14.7879
2022-05-25 09:14:17,268 | split   1/  1 |   300/  340 batches | ms/batch 308.54 | loss 2.4596 | ppl 11.6998
2022-05-25 09:14:30,424 best split so far
2022-05-25 09:14:30,425 best loss so far 2.29728925
2022-05-25 09:14:30,652 ('\nเรื่องเป็นเรา เรียวโตรดีเล่นใหทงรีวิวเองี้คาดว่าว่าตลอมติ่งทิอะไรแล้วก็ไปต่อกับตา รู้สึกว่าฝีทวิตกเริ่มควะเอาใจไว้าอีกญี่ดูโอเศจอหล่างให้พี่ข้างสือมันก็ได้ #ลืงๆ ##CngOTOOtMOkATP #BottaSf ๑hasdenyuiload ชิดแน่นเพื่อนภาษาสวมหนึงเราก็ไม่พอก็แบบแอนนี้นเเราร้องที่พี่เราใช่ใจมาเราห้องแบ

RuntimeError: DataLoader worker (pid(s) 49839) exited unexpectedly

In [ ]:
# maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-sample", name="VISTEC-sample")
maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-2021", name="VISTEC")
train_model(maindataset, alphabetized=True)

In [ ]:
"DONE"

In [ ]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings
from flair.embeddings import StackedEmbeddings

# glove_embedding = WordEmbeddings('glove')
alphabetized = True
model_dir_suffix = ""
if alphabetized:
    model_dir_suffix = "_alph"
    
flair_embedding_forward = FlairEmbeddings(f"./Models/fwdLM{model_dir_suffix}/best-lm.pt")
flair_embedding_backward = FlairEmbeddings(f"./Models/bkwLM{model_dir_suffix}/best-lm.pt")

stacked_embeddings = StackedEmbeddings([flair_embedding_forward, flair_embedding_backward,])

In [ ]:
from flair.data import Sentence
sentence = Sentence(["ฉัน", "รัก", "แมว"])

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding.shape)

# Intrinsic Evaluation

In [18]:
import sys
sys.path.append('../Libs/word-embeddings-benchmarks/')

In [19]:
import logging, sys
import scipy.stats
from six import iteritems
from web.datasets.similarity import fetch_MEN, fetch_WS353, fetch_SimLex999
from web.datasets.similarity import fetch_TWS65, fetch_thai_wordsim353, fetch_thai_semeval2017_task2, fetch_thai_simlex999
from web.embeddings import load_embedding
# from web.evaluate import evaluate_similarity

In [20]:
import sklearn

In [21]:
import numpy as np
from scipy import spatial
    
def evaluate_similarity(wv, X, y, preprocess=None):
    
    missing_words, found_words, oov_vecs_created, index = 0, 0, 0, 0
    word_pair_oov_indices = []
    info_oov_words = {}
    info_created_words = {}

    ## For all words in the datasets, check if the are OOV? 
    ## Indices of word-pairs with a OOV word are stored in word_pair_oov_indices
    
    nwords = 0
    for query in X:
        for query_word in query:
            found_words += 1
            nwords += 1
        index += 1

    # print(f"Missing Word: {missing_words} words ({missing_words*100/nwords:.2f}%)")
    

    # The original code; for all OOV; it will be replaced by average vector
    # mean_vector = np.mean(w.vectors, axis=0, keepdims=True)
    # A = np.vstack(w.get(word, mean_vector) for word in X[:, 0])
    # B = np.vstack(w.get(word, mean_vector) for word in X[:, 1])
    # scores = np.array([v1.dot(v2.T)/(np.linalg.norm(v1)*np.linalg.norm(v2)) for v1, v2 in zip(A, B)])
    
    scores = []
    for w in X:
        vecA = wv.get_vector(w[0])
        vecB = wv.get_vector(w[1])
        s = 1 - spatial.distance.cosine(vecA, vecB)
        scores.append(s)
        
#     A = np.vstack(w[preprocess(word)] for word in )
#     B = np.vstack(w[preprocess(word)] for word in X[:, 1])
#     scores = np.array([v1.dot(v2.T)/(np.linalg.norm(v1)*np.linalg.norm(v2)) for v1, v2 in zip(A, B)])


    # wohlg: original version only returned Spearman 
    # wohlg: we added Pearson and other information 
    result = {
        'spearmanr': scipy.stats.spearmanr(scores, y).correlation,
        'pearsonr':  scipy.stats.pearsonr(scores, y)[0],
        'num_oov_word_pairs': len(word_pair_oov_indices),
        'num_found_words': found_words,
        'num_missing_words': missing_words,
        "num_word_pairs": nwords
    }

    return result


In [22]:
import fasttext


In [23]:
tasks = {
    "TH-WS353": fetch_thai_wordsim353(),
    "TH-SemEval2017T2": fetch_thai_semeval2017_task2(),
    "TH-SimLex999": fetch_thai_simlex999(),
    "TWS65": fetch_TWS65()
}

# Print sample data
for name, data in iteritems(tasks):
    print("Sample data from {}: pair \"{}\" and \"{}\" is assigned score {}".format(name, data.X[0][0], data.X[0][1], data.y[0]))

def eval_word_sim(wv, verbose=True):
    # Calculate results using helper function for the various word similarity datasets
    results = {}
    for name, data in iteritems(tasks):
        result = evaluate_similarity(wv, data.X, data.y)

    #     hm = scipy.stats.hmean([result['spearmanr'], result['pearsonr']])
        perc_oov_words = 100 * (result['num_missing_words'] / (result['num_found_words'] + float(result['num_missing_words'])))

        # Spearman: evaluate a monotonic relationship between two variables based on the ranked values for each variable rather than the raw data.
        # Pearson : measures the linear correlation between two variables X and Y
        if verbose:
            print(f"Dataset {name}: Spearman: {result['spearmanr']:4.3f}")
        results[name] = result['spearmanr']
    return results


Sample data from TH-WS353: pair "ความรัก" and "เพศ" is assigned score 12.2
Sample data from TH-SemEval2017T2: pair "จูล" and "ยานอวกาศ" is assigned score 1.9
Sample data from TH-SimLex999: pair "เก่า" and "ใหม่" is assigned score 4.38
Sample data from TWS65: pair "แก้ว" and "ข้ารับใช้" is assigned score 0.116


In [24]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.data import Sentence

class WordVector:
    def __init__(self, alphabetized, fwd=True, bkw=True):
        model_dir_suffix = ""
        if alphabetized:
            model_dir_suffix = "_alph"
        
        embs = []
        if fwd:
            flair_embedding_forward = FlairEmbeddings(f"./Models/fwdLM{model_dir_suffix}/best-lm.pt")
            embs.append(flair_embedding_forward)
        
        if bkw:
            flair_embedding_backward = FlairEmbeddings(f"./Models/bkwLM{model_dir_suffix}/best-lm.pt")
            embs.append(flair_embedding_backward)
        
        self.alphabetized = alphabetized
        self.stacked_embeddings = StackedEmbeddings(embs)

        
    def get_vector(self, word):
        if self.alphabetized:
#             print(word, "".join(custom_alphabetize(word)))
            word = "".join(custom_alphabetize(word))
            
        sentence = Sentence([word])
        self.stacked_embeddings.embed(sentence)

        for token in sentence:
            return token.embedding

wv = WordVector(alphabetized=True)
wordsim = eval_word_sim(wv)


Dataset TH-WS353: Spearman: 0.152
Dataset TH-SemEval2017T2: Spearman: 0.197
Dataset TH-SimLex999: Spearman: 0.163
Dataset TWS65: Spearman: 0.139


In [25]:
wv = WordVector(alphabetized=False)
wordsim = eval_word_sim(wv)

Dataset TH-WS353: Spearman: 0.222
Dataset TH-SemEval2017T2: Spearman: 0.175
Dataset TH-SimLex999: Spearman: 0.180
Dataset TWS65: Spearman: 0.014


### w/ Alphabetized
* Dataset TH-WS353: Spearman: 0.152
* Dataset TH-SemEval2017T2: Spearman: 0.197
* Dataset TH-SimLex999: Spearman: 0.163
* Dataset TWS65: Spearman: 0.139

### w/o Alphabetized
* Dataset TH-WS353: Spearman: 0.222
* Dataset TH-SemEval2017T2: Spearman: 0.175
* Dataset TH-SimLex999: Spearman: 0.180
* Dataset TWS65: Spearman: 0.014

## Target on Fasttext on Word-level

W/o OOV
* Dataset TH-WS353: Spearman: 0.182
* Dataset TH-SemEval2017T2: Spearman: 0.175
* Dataset TH-SimLex999: Spearman: 0.201
* Dataset TWS65: Spearman: 0.203

With OOV
* Dataset TH-WS353: Spearman: 0.347
* Dataset TH-SemEval2017T2: Spearman: 0.371
* Dataset TH-SimLex999: Spearman: 0.410
* Dataset TWS65: Spearman: 0.252

Cite: [Word Similarity Datasets for Thai: Construction and Evaluation](https://ieeexplore.ieee.org/document/8851127)

In [ ]:
"DONE"